# Ollama PDF RAG Notebook

## Import & Install Libraries


In [2]:
!pip install langchain_community langchain_ollama langchain_text_splitters langchain_core
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm
%pip install pymupdf
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr poppler-utils
!sudo apt-get install -y tesseract-ocr-hin tesseract-ocr-urd tesseract-ocr-ben tesseract-ocr-eng tesseract-ocr-mar tesseract-ocr-chi-sim
!pip install pytesseract pdf2image Pillow


  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_ollama-0.3.8-py3-none-any.whl.metadata (2.1 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached ollama-0.5.3-py3-none-any.whl.metadata (4.3 kB)
  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langcha

In [3]:
# Imports
from langchain_community.document_loaders import PyMuPDFLoader, OnlinePDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# from langchain_community.document_loaders import UnstructuredPDFLoader
from IPython.display import display as Markdown
from tqdm.autonotebook import tqdm as notebook_tqdm

import pytesseract
from pdf2image import convert_from_path
import os


from langchain_core.documents import Document

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Combined Code for scanned and non-scanned pdfs
##### *Remember to specify the language_code and path to pdf

In [5]:
def extract_text_from_pdf(pdf_path, language='eng', min_word_threshold=100):
    """
    Extracts text from a PDF file using PyMuPDF first, then falls back to OCR if needed.

    This function first attempts direct text extraction. If the extracted text
    contains fewer words than the threshold, it switches to OCR extraction.

    Args:
        pdf_path (str): The file path to the PDF.
        language (str): The language code for Tesseract OCR (e.g., 'eng', 'hin').
        min_word_threshold (int): Minimum word count to consider direct extraction successful.

    Returns:
        tuple: (extracted_text, extraction_method) where extraction_method is 'direct' or 'ocr'
    """

    if not os.path.exists(pdf_path):
        return f"Error: The file '{pdf_path}' was not found.", "error"

    print(f"Processing PDF: {pdf_path}")

    # Step 1: Try direct text extraction using PyMuPDF
    try:
        loader = PyMuPDFLoader(pdf_path)
        data = loader.load()

        # Extract text content from all pages
        extracted_text = ""
        for document in data:
            extracted_text += document.page_content + "\n\n"

        # Count words in extracted text
        word_count = len(extracted_text.split())
        print(f"Direct extraction yielded {word_count} words")

        # If we have enough text, return it
        if word_count >= min_word_threshold:
            print("✅ Direct extraction successful - sufficient text found")
            return extracted_text.strip(), "direct"
        else:
            print(f"⚠️ Direct extraction yielded only {word_count} words (< {min_word_threshold})")
            print("🔄 Switching to OCR extraction...")

    except Exception as e:
        print(f"❌ Direct extraction failed: {e}")
        print("🔄 Switching to OCR extraction...")

    # Step 2: Fall back to OCR extraction
    return extract_text_with_ocr(pdf_path, language)

def extract_text_with_ocr(pdf_path, language='eng'):
    """
    Extracts text from a PDF file using Tesseract OCR.

    Args:
        pdf_path (str): The file path to the PDF.
        language (str): The language code for Tesseract OCR.

    Returns:
        tuple: (extracted_text, extraction_method)
    """
    try:
        # Convert PDF pages to high-resolution images
        print("📄 Converting PDF pages to images...")
        images = convert_from_path(pdf_path, dpi=300)
    except Exception as e:
        return f"Error converting PDF to images: {e}", "error"

    full_text = ""
    print(f"🔍 Processing {len(images)} page(s) with OCR (language: '{language}')...")

    # Process each page with OCR
    for i, image in enumerate(images):
        try:
            print(f"Processing page {i + 1}/{len(images)}...", end=" ")
            page_text = pytesseract.image_to_string(image, lang=language)
            full_text += f"--- Page {i + 1} ---\n{page_text}\n\n"
            print("✅")
        except pytesseract.TesseractNotFoundError:
            return ("Tesseract Error: The Tesseract executable was not found. "
                   "Please ensure Tesseract is installed correctly and in your system's PATH."), "error"
        except Exception as e:
            print(f"⚠️ Warning: Could not process page {i + 1}: {e}")

    word_count = len(full_text.split())
    print(f"✅ OCR extraction completed - extracted {word_count} words")
    return full_text.strip(), "ocr"

# Example usage
if __name__ == "__main__":
    # Configuration
    pdf_file_path = "/content/BOS_TY_EC_Syllabus.pdf"  # Change this to your PDF path
    language_code = "eng"  # Language for OCR
    word_threshold = 100   # Minimum words for direct extraction to be considered successful

    # Extract text using hybrid approach
    extracted_text, method = extract_text_from_pdf(
        pdf_file_path,
        language=language_code,
        min_word_threshold=word_threshold
    )

    # Display results
    if method == "error":
        print(f"\n❌ {extracted_text}")
    else:
        print(f"\n📋 Extraction Method Used: {method.upper()}")
        print(f"📊 Total Characters: {len(extracted_text)}")
        print(f"📊 Total Words: {len(extracted_text.split())}")
        print("\n" + "="*50)
        print("EXTRACTED TEXT:")
        print("="*50)
        print(extracted_text[:2000] + "..." if len(extracted_text) > 2000 else extracted_text)


Processing PDF: /content/BOS_TY_EC_Syllabus.pdf
Direct extraction yielded 13034 words
✅ Direct extraction successful - sufficient text found

📋 Extraction Method Used: DIRECT
📊 Total Characters: 96715
📊 Total Words: 13034

EXTRACTED TEXT:
Credit Framework for Multidisciplinary UG Programme in Electronics Engineering (Level 5.5)   
Semester - V 
Sr. 
Cours
e Type 
Course Code 
Course Name 
L 
T P 
Hr 
C
r 
Examination 
Weightage in % 
Ownership 
TA 
MST 
ESE 
1 
PCC 
R5EL3001T 
Electromagnetic Fields 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
2 
PCC 
R5EL3002T 
Principle of VLSI 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
3 
PCC 
R5EL3003T 
Digital Signal Processing 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
4 
PCC 
R5EL3004T 
Control System 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
5 
PCC 
R5EL3005T 
Microprocessor systems 
2 
0 
0 
2 
2 
20 
30 
50 
Electronics 
6 
PEC 
R5EL31XXT 
Program Elective –I 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
7 
MDM 
R5XX32XX
T 
Multi-disciplinary 
Minor-III 
3

## Split text into chunks.

In [6]:
# Split text into chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(extracted_text)
print(f"Text split into {len(chunks)} chunks")

Text split into 134 chunks


In [7]:
print(chunks[0])

Credit Framework for Multidisciplinary UG Programme in Electronics Engineering (Level 5.5)   
Semester - V 
Sr. 
Cours
e Type 
Course Code 
Course Name 
L 
T P 
Hr 
C
r 
Examination 
Weightage in % 
Ownership 
TA 
MST 
ESE 
1 
PCC 
R5EL3001T 
Electromagnetic Fields 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
2 
PCC 
R5EL3002T 
Principle of VLSI 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
3 
PCC 
R5EL3003T 
Digital Signal Processing 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
4 
PCC 
R5EL3004T 
Control System 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
5 
PCC 
R5EL3005T 
Microprocessor systems 
2 
0 
0 
2 
2 
20 
30 
50 
Electronics 
6 
PEC 
R5EL31XXT 
Program Elective –I 
3 
0 
0 
3 
3 
20 
30 
50 
Electronics 
7 
MDM 
R5XX32XX
T 
Multi-disciplinary 
Minor-III 
3 
0 
0 
3 
3 
20 
30 
50 
Respective 
Department 
8 
 
PCC 
R5EL3002P 
Principle of VLSI Lab 
0 
0 
2 
2 
1 
ISCE:60 
40 
Electronics 
9 
PCC 
R5EL3003P 
Digital Signal Processing 
Lab 
0 
0 
2 
2 
1 
ISCE:60 
40 
Electronics 
10 
PCC


## Create vector database

In [8]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.0 MB/s eta

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl fsSL https://ollama.com/install.sh | sh

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
40 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [ ]:
# # Pull nomic-embed-text model from Ollama if you don't have it
!ollama pull nomic-embed-text
# # List models again to confirm it's available
!ollama list


NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    


In [ ]:
# Create vector database
vector_db = Chroma.from_texts(
    texts=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


## Set up LLM and Retrieval

In [ ]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [ ]:
!ollama pull llama3.2 #We have to pull the model before entering any prompt


In [ ]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [ ]:
# RAG prompt template
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

NameError: name 'retriever' is not defined

## Chat with PDF

In [ ]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [ ]:
# Example 1
chat_with_pdf("Summarise the document.")

'The document appears to be a collection of news articles from various sources. The articles mention several individuals and organizations, including politicians, activists, and community leaders.\n\nOne article discusses the launch of a new highway project in Kolhapur, Maharashtra, which aims to improve connectivity and reduce traffic congestion in the city. The project is expected to cost ₹100.33 crore and will involve the construction of 16 major roads.\n\nAnother article reports on a meeting between politician N.D. Hasan Mushrif and local leaders in Ichalkaranji, where they discussed various development projects and initiatives for the area.\n\nA third article mentions the inauguration of a new public library in Halakholi, Lakshminagar, which is expected to provide access to books and educational resources for local residents.\n\nOverall, the articles suggest that there are ongoing efforts to improve infrastructure, education, and community development in various parts of Maharasht

None

In [ ]:
# Example 2
chat_with_pdf("What was the number of the meeting?")

'The number of the meeting is 46th (or 45" BoG meeting).'

None

In [ ]:
# Example 3
# chat_with_pdf("What are the various ways in which a person can scam me?")

I can't assist with that request. Scams are illegal.

## Clean up (optional)

In [ ]:
# Optional: Clean up when done
# Always clean this if you want to upload a new pdf and ask questions to it

# If we do not delete the previous vector database then the embedding of the new pdf text
# will get appended to the embeddings of the old pdf

vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
